# ProcessBench 的四个数据集

In [1]:
import os
os.environ['HF_HOME'] = '/root/autodl-tmp/hf-mirror'
os.environ['HF_ENDPOINT'] = "https://hf-mirror.com"
from datasets import load_dataset

ds = load_dataset("Qwen/ProcessBench")

/root/miniconda3/envs/prm_openrlhf/lib/python3.10/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


README.md: 0.00B [00:00, ?B/s]

gsm8k.json: 0.00B [00:00, ?B/s]

math.json: 0.00B [00:00, ?B/s]

olympiadbench.json: 0.00B [00:00, ?B/s]

omnimath.json: 0.00B [00:00, ?B/s]

Generating gsm8k split: 0 examples [00:00, ? examples/s]

Generating math split: 0 examples [00:00, ? examples/s]

Generating olympiadbench split: 0 examples [00:00, ? examples/s]

Generating omnimath split: 0 examples [00:00, ? examples/s]

In [ ]:
datasets = ["gsm8k", "math", "olympiadbench", "ominimath"]
formatted_data = []
for dataset in datasets:
    sub_ds = ds[dataset]
    for i in range(len(sub_ds)):
        _id = sub_ds[i]['id']
        problem = sub_ds[i]['problem']
        steps = sub_ds[i]['steps']
        label = sub_ds[i]['label']
        final_answer_correct = sub_ds[i]['final_answer_correct']

        _label = ""
        _input = ""
        _label += problem + "\n"
        _input += problem + "\n"
        for idx, step in enumerate(steps):
            tag = " [POS]" if idx < label else " [NEG]"
            _label += step + tag
            _input += step + " [PRM]"
            if idx != len(steps) - 1:
                _label += "\n"
                _input += "\n"
        
        value = ["[POS]"] * label + ["[NEG]"] * (len(steps) - label)
        # print("ID: " + _id)
        # print("Problem: \n" + problem)
        # print("Label: \n" + _label)
        # print("Input: \n" + _input)
        # print("Value: \n" + str(value))
        formatted_data.append({
            "id": _id,
            "problem": problem,
            "label": _label,
            "input": _input,
            "value": value,
            "final_answer_correct": final_answer_correct,
            "dataset": dataset
        })

# 将 formatted_data 保存为 JSON 文件
import json
with open("formatted_processbench_data.json", "w") as f:
    json.dump(formatted_data, f, indent=4)

# 将 formatted_data 上传到 Hugging Face Datasets
from datasets import Dataset
formatted_dataset = Dataset.from_list(formatted_data)
formatted_dataset.push_to_hub("weepcat/ProcessBench")

ID: gsm8k-0
Problem: 
Sue lives in a fun neighborhood.  One weekend, the neighbors decided to play a prank on Sue.  On Friday morning, the neighbors placed 18 pink plastic flamingos out on Sue's front yard.  On Saturday morning, the neighbors took back one third of the flamingos, painted them white, and put these newly painted white flamingos back out on Sue's front yard.  Then, on Sunday morning, they added another 18 pink plastic flamingos to the collection. At noon on Sunday, how many more pink plastic flamingos were out than white plastic flamingos?
Label: 
Sue lives in a fun neighborhood.  One weekend, the neighbors decided to play a prank on Sue.  On Friday morning, the neighbors placed 18 pink plastic flamingos out on Sue's front yard.  On Saturday morning, the neighbors took back one third of the flamingos, painted them white, and put these newly painted white flamingos back out on Sue's front yard.  Then, on Sunday morning, they added another 18 pink plastic flamingos to the c

In [ ]:
ds['gsm8k'][0]['problem']

{'id': 'gsm8k-0',
 'generator': 'Qwen2-7B-Instruct',
 'problem': "Sue lives in a fun neighborhood.  One weekend, the neighbors decided to play a prank on Sue.  On Friday morning, the neighbors placed 18 pink plastic flamingos out on Sue's front yard.  On Saturday morning, the neighbors took back one third of the flamingos, painted them white, and put these newly painted white flamingos back out on Sue's front yard.  Then, on Sunday morning, they added another 18 pink plastic flamingos to the collection. At noon on Sunday, how many more pink plastic flamingos were out than white plastic flamingos?",
 'steps': ['To find out how many more pink plastic flamingos were out than white plastic flamingos at noon on Sunday, we can break down the problem into steps. First, on Friday, the neighbors start with 18 pink plastic flamingos.',
  "On Saturday, they take back one third of the flamingos. Since there were 18 flamingos, \\(1/3 \\times 18 = 6\\) flamingos are taken back. So, they have \\(18 -

# 结合 Step-Level Beam Search 和 Best-of-N 进行评估 (GSM8K, MATH, Olympiad Bench, MMLU  STEM)

In [17]:
import os
os.environ['HF_HOME'] = '/root/autodl-tmp/hf-mirror'
os.environ['HF_ENDPOINT'] = "https://hf-mirror.com"
from datasets import load_dataset

ds = load_dataset("openai/gsm8k", "main", split="test")

In [ ]:
# 提取 question 和 answer
formatted_data = []
import re
# #### 18
# parttern 是答案在 #### 后面的部分
parttern = r"#### (.*)"
for i in range(len(ds)):
    problem = ds[i]['question']
    answer = ds[i]['answer']
    # 使用正则表达式提取答案
    match = re.search(parttern, answer, re.DOTALL)
    if match:
        extracted_answer = match.group(1).strip()
    else:
        extracted_answer = answer.strip()
    formatted_data.append({
        "problem": problem,
        "answer": extracted_answer
    })

# # 将 formatted_data 上传到 Hugging Face Datasets
from datasets import Dataset
formatted_dataset = Dataset.from_list(formatted_data)
formatted_dataset.push_to_hub("weepcat/gsm8k", split="test")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md:   0%|          | 0.00/307 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/weepcat/gsm8k/commit/e93200057d2e063343c5393c32de89a31c9c198c', commit_message='Upload dataset', commit_description='', oid='e93200057d2e063343c5393c32de89a31c9c198c', pr_url=None, repo_url=RepoUrl('https://hf-mirror.com/datasets/weepcat/gsm8k', endpoint='https://hf-mirror.com', repo_type='dataset', repo_id='weepcat/gsm8k'), pr_revision=None, pr_num=None)

In [21]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceH4/MATH-500", split="test")


In [22]:
# 提取 question 和 answer
formatted_data = []
for i in range(len(ds)):
    problem = ds[i]['problem']
    answer = ds[i]['answer']
    formatted_data.append({
        "problem": problem,
        "answer": answer
    })

# # 将 formatted_data 上传到 Hugging Face Datasets
from datasets import Dataset
formatted_dataset = Dataset.from_list(formatted_data)
formatted_dataset.push_to_hub("weepcat/MATH-500", split="test")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/weepcat/MATH-500/commit/e3a3a785b6b9dcdd699891632e8eec1427b5246f', commit_message='Upload dataset', commit_description='', oid='e3a3a785b6b9dcdd699891632e8eec1427b5246f', pr_url=None, repo_url=RepoUrl('https://hf-mirror.com/datasets/weepcat/MATH-500', endpoint='https://hf-mirror.com', repo_type='dataset', repo_id='weepcat/MATH-500'), pr_revision=None, pr_num=None)

In [23]:
from datasets import load_dataset

ds = load_dataset("math-ai/minervamath", split="test")


README.md:   0%|          | 0.00/246 [00:00<?, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating test split:   0%|          | 0/272 [00:00<?, ? examples/s]

In [24]:
# 提取 question 和 answer
formatted_data = []
for i in range(len(ds)):
    problem = ds[i]['question']
    answer = ds[i]['answer']
    formatted_data.append({
        "problem": problem,
        "answer": answer
    })

# # 将 formatted_data 上传到 Hugging Face Datasets
from datasets import Dataset
formatted_dataset = Dataset.from_list(formatted_data)
formatted_dataset.push_to_hub("weepcat/minervamath", split="test")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/weepcat/minervamath/commit/230467124b3368d1b683da08718fde8d52dd3667', commit_message='Upload dataset', commit_description='', oid='230467124b3368d1b683da08718fde8d52dd3667', pr_url=None, repo_url=RepoUrl('https://hf-mirror.com/datasets/weepcat/minervamath', endpoint='https://hf-mirror.com', repo_type='dataset', repo_id='weepcat/minervamath'), pr_revision=None, pr_num=None)

In [25]:
from datasets import load_dataset

ds = load_dataset("MARIO-Math-Reasoning/Gaokao2023-Math-En", split="train")


README.md:   0%|          | 0.00/292 [00:00<?, ?B/s]

Gaokao2023-Math-En.jsonl: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [26]:
# 提取 question 和 answer
formatted_data = []
for i in range(len(ds)):
    problem = ds[i]['question']
    answer = ds[i]['answer']
    formatted_data.append({
        "problem": problem,
        "answer": answer
    })

# # 将 formatted_data 上传到 Hugging Face Datasets
from datasets import Dataset
formatted_dataset = Dataset.from_list(formatted_data)
formatted_dataset.push_to_hub("weepcat/Gaokao2023-Math-En", split="test")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/weepcat/Gaokao2023-Math-En/commit/98940c9dbb5048244ef627a3edd126ff1b3a4938', commit_message='Upload dataset', commit_description='', oid='98940c9dbb5048244ef627a3edd126ff1b3a4938', pr_url=None, repo_url=RepoUrl('https://hf-mirror.com/datasets/weepcat/Gaokao2023-Math-En', endpoint='https://hf-mirror.com', repo_type='dataset', repo_id='weepcat/Gaokao2023-Math-En'), pr_revision=None, pr_num=None)

In [ ]:
# qwen2.5-7b-instruct
# qwen2.5-math-7b-instruct
from openai import OpenAI, AsyncOpenAI
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# completion = client.chat.completions.create(
#     # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
#     model="qwen-plus",
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": "你是谁？"},
#     ]
# )
# print(completion.model_dump_json())